In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(ggplot2)
library(readr)
library(gridExtra)
library(dplyr)

ig_80 <- read.csv("/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/sorted_by_ig1030_time_cross_80.csv", sep="\t")
ig_origin <- read.csv("/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/sorted_by_ig1030_time_cross.csv", sep="\t")


Attaching package: ‘dplyr’

The following object is masked from ‘package:gridExtra’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [ ]:
%%R
library(data.table)

ig_origin_dt <- as.data.table(ig_origin)
ig_80_dt <- as.data.table(ig_80)

ig_origin_dt <- ig_origin_dt[order(-ig_1), .SD[1], by = start][, index1 := frank(-ig_1, ties.method = "first")]
ig_80_dt <- ig_80_dt[order(-ig_1), .SD[1], by = start][, index2 := frank(-ig_1, ties.method = "first")]

table_dt <- ig_origin_dt[, .(start, index1)]
table_dt <- merge(table_dt, ig_80_dt[, .(start, index2)], by = "start", all.x = TRUE)

print(head(table_dt))


Key: <start>
           start index1 index2
          <char>  <int>  <int>
1: R-MMU-1006143    939    803
2: R-MMU-1011576   3632   2008
3: R-MMU-1013012   2548    434
4: R-MMU-1013013   2247    574
5: R-MMU-1022127   3344   3711
6: R-MMU-1028788   3986   4668


data.table 1.16.0 using 8 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [ ]:
%%R
columns_to_compare <- c("index2")
file_names <- c("ig_80")

pearson_correlations <- list()

for (i in seq_along(columns_to_compare)) {
  result <- cor.test(table_dt$index1, table_dt[[columns_to_compare[i]]], method = "pearson")
  
  pearson_correlations[[i]] <- list(
    comparison = paste("ig_origin_vs", file_names[i], sep = "_"),
    rho = result$estimate,
    p_value = result$p.value,
    t_value = result$statistic
  )
}

print("------pearson_correlations-------")
for (res in pearson_correlations) {
  print(paste0(res$comparison, ": rho = ", round(res$rho, 4), ", p-value = ", format(res$p_value, scientific = TRUE), ", t-value = ", res$t_value))
}


[1] "------pearson_correlations-------"
[1] "ig_origin_vs_ig_80: rho = 0.3954, p-value = 7.099827e-178, t-value = 29.6911465979497"


In [ ]:
%%R
columns_to_compare <- c("index2")
file_names <- c("ig_80")

spearman_correlations <- list()

for (i in seq_along(columns_to_compare)) {
  result <- cor.test(table_dt$index1, table_dt[[columns_to_compare[i]]], method = "spearman")
  
  spearman_correlations[[i]] <- list(
    comparison = paste("ig_origin_vs", file_names[i], sep = "_"),
    rho = result$estimate,
    p_value = result$p.value,
    S_value = result$statistic
  )
}

print("------spearman_correlations-------")
for (res in spearman_correlations) {
  print(paste0(res$comparison, ": rho = ", round(res$rho, 4), ", p-value = ", format(res$p_value, scientific = TRUE), ", S-value = ", res$S_value))
}


[1] "------spearman_correlations-------"
[1] "ig_origin_vs_ig_80: rho = 0.3954, p-value = 7.099827e-178, S-value = 10853280962"
